### 2018/2019 - Task List 10

1. Implement Naive Bayes classifier with pyro
    - create apropriate parameters (mean and std for a and b, sigma - noise)
    - provide optimization procedure
    - check appropriateness of implemented method with selected dataset


# Required imports

In [1]:
%matplotlib inline
import pyro
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyro.optim as optim
import pyro.distributions as dist
from torch.distributions import constraints
from tqdm import tqdm
import seaborn as sns
from matplotlib import animation, rc
from IPython.display import HTML
import torch.nn as nn
from functools import partial
import pandas as pd
from pyro.contrib.autoguide import AutoDiagonalNormal
from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, TracePredictive

In [2]:
pyro.set_rng_seed(1)
pyro.enable_validation(True)

## Solutions

In [3]:
def read_dataset():    
    data_seeds = pd.read_csv('./data_sets/seeds.data', sep="\t", header=None)
    data_seeds.columns = ["Area A", "Perimeter P", "Compactness C = 4*pi*A/P^2", "Length of kernel",
                      "Width of kernel", "Asymmetry coefficient", "Length of kernel groove", "decision"]
    return data_seeds

def get_attributes_mi_sigma(data_set):
    mean = data_seeds.mean(axis=0).tolist()[:-1]
    std = data_seeds.std(axis=0).tolist()[:-1]
    matrix = [[] for _ in range(len(mean))]

    for idx in range(len(mean)):
        # mean, std, sigma <- for every attribute in dataset
        matrix[idx] = [idx, mean[idx], std[idx], pyro.sample("mean_", dist.Uniform(0., 5.))]
        
    return matrix

# def init_attr_priors(data_set):

# assuming normal distributuion of continuous attributes, we can use pdf instead of discretization
def probability_density_function(mean, std, value):
    # shouldn't happen
    if std == 0:
        std = 0.001
#         return .001

    return 1 / (std * np.sqrt(2 * np.pi)) * np.exp(- (value - mean) ** 2 / (2 * std ** 2))

In [4]:
class RegressionModel(nn.Module):
    def __init__(self):
        # p = number of features
        super(RegressionModel, self).__init__()
        self.a = nn.Linear(1, 1, bias=False)
        self.b = nn.Parameter(torch.tensor(1.))

    def forward(self, x):
        # todo check it
        return self.a(x) + self.b
    
def row_sampling(row, data_set):
    noise = row[3]
    label = "mean_" + str(row[0])
    mean = pyro.sample("mean_", dist.Normal(row[1], noise))
    std = pyro.sample("std_" + str(row[0]), dist.Normal(row[2], noise))
    sigma = pyro.sample("sigma_" + str(row[0]),  dist.Uniform(0., 5.))
    return [mean, std, sigma]
    
def create_probabilistic_model():
    regression_model=RegressionModel()
    def probabilistic_model(data_set):
        priors_pd = pd.DataFrame(data=priors, columns=["idx", "mean", "std", "noise"])
    
    # todo check if it work proper
    # if not - work around could be adding new values to new tmp columns
    # then move value from tmp to original and delete tmp
    priors_pd[["mean", "std", "noise"]] = priors_pd.apply(
        lambda row: pd.Series(row_sampling(row, data_seeds)), axis=1
    )
        
#         a = pyro.sample("a", dist.Normal(8., 1000.))
#         b_a = pyro.sample("bA", dist.Normal(0., 1.))
#         sigma = pyro.sample("sigma",  dist.Uniform(0., 10.))        
    
    with pyro.plate("map", len(x_data)):
#             todo update matrix
        pyro.sample("obs", matrix, obs=y_data)
        return mean
    return probabilistic_model

def guide(attributes_mi_sigma):
#     a_loc = pyro.param('w_loc', torch.tensor(0.))
#     a_scale = pyro.param('w_scale', torch.tensor(1.),
#                         constraint=constraints.positive)
#     b_loc = pyro.param('b_loc', torch.tensor(0.))
#     b_scale = pyro.param('b_scale', torch.tensor(1.),
#                         constraint=constraints.positive)  
#     sigma_loc = pyro.param('sigma_loc', torch.tensor(1.),
#                              constraint=constraints.positive)
    
#     a = pyro.sample("a", dist.Normal(a_loc, a_scale))
#     b_a = pyro.sample("bA", dist.Normal(b_loc, b_scale))
#     sigma = pyro.sample("sigma", dist.Normal(sigma_loc, torch.tensor(0.05)))
    mean = a * x_data + b_a


def train():
    pyro.clear_param_store()
    num_iterations = 3000 #8000
    model = create_probabilistic_model()
#     guide = AutoDiagonalNormal(model)
    optim = pyro.optim.Adam({"lr": 0.005})
    svi = pyro.infer.SVI(model, guide, optim, loss=pyro.infer.Trace_ELBO(), num_samples=count)
    t=tqdm(range(num_iterations))
    for j in t:
        loss = svi.step(mcg_data, gvh_data)
        t.set_postfix(loss=loss)
    return (model, svi)

# def get_marginal(traces, sites):
#     empirical_marginal=EmpiricalMarginal(traces, sites)
#     return empirical_marginal._get_samples_and_weights()[0].detach().cpu().numpy()

# def summary(traces, sites):
#     marginal = get_marginal(traces, sites)
#     site_stats = {}
#     for i in range(marginal.shape[1]):
#         site_name = sites[i]
#         site_marginal=marginal[:, i]
#         marginal_site = pd.DataFrame(site_marginal).transpose()
#         describe = partial(pd.Series.describe, percentiles=[.05, 0.25, 0.5, 0.75, 0.95])
#         site_stats[site_name] = marginal_site.apply(describe, axis=1) \
#             [["mean", "std", "5%", "25%", "50%", "75%", "95%"]]
#     return site_stats

# def wrapped_model(x_data, y_data):
#     model_result=probabilistic_model(x_data, y_data)
#     pyro.sample("prediction", pyro.distributions.Delta(model_result))

In [5]:
data_seeds = read_dataset()
count = len(data_seeds)
priors = get_attributes_mi_sigma(data_set=data_seeds)
priors_pd = pd.DataFrame(data=priors, columns=["idx", "mean", "std", "noise"])
print(priors_pd.iloc[0][0])
# priors_pd[["mean", "std", "noise"]]

print(priors_pd)
priors_pd[["mean", "std", "noise"]] = priors_pd.apply(
    lambda row: pd.Series(row_sampling(row, data_seeds)), axis=1
)
print(priors_pd)
# tst = "mean_" + str(priors_pd.iloc[0][0])
# print(type(tst))
# print(type(priors_pd.iloc[0][3]))
# dist.Normal(3, 0.01)
# tst = pyro.sample("mean_", dist.Uniform(0., 5.))
# print(tst)
# row_sampling(priors_pd.iloc[0], data_seeds)

# check parameters
# print("Check parameters:")
# for name, value in pyro.get_param_store().items():
#     print(name, pyro.param(name))

0
   idx       mean       std           noise
0    0  14.847524  2.909699  tensor(3.7882)
1    1  14.559286  1.305959  tensor(1.3966)
2    2   0.870999  0.023629  tensor(2.0153)
3    3   5.628533  0.443063  tensor(3.6734)
4    4   3.258605  0.377714  tensor(0.1464)
5    5   3.700201  1.503557  tensor(3.9993)
6    6   5.408071  0.491480  tensor(1.9857)
   idx             mean             std           noise
0    0   tensor(9.0790)  tensor(4.3556)  tensor(2.8624)
1    1  tensor(15.6004)  tensor(1.2011)  tensor(0.9901)
2    2  tensor(-0.6661)  tensor(2.8390)  tensor(3.9903)
3    3   tensor(7.5549)  tensor(4.1741)  tensor(4.8714)
4    4   tensor(3.1969)  tensor(0.3029)  tensor(4.6953)
5    5   tensor(2.5924)  tensor(3.4115)  tensor(1.4423)
6    6   tensor(9.6038)  tensor(0.4836)  tensor(1.1729)


In [9]:
apriori = pyro.sample("apriori", dist.Beta(1, 1))
abc = pyro.sample("abs", dist.Normal(3, 2))

with pyro.plate("map", size=2):
    # c=pyro.sample("", Categorical(abc)) # wym: 1x2
    a=pyro.sample("", dist.Normal([apriori])) # wym: 3x2
    pdf = PDF(x_data) # wym: n x 3

    pdf_c = abc * pdf # wym: n x 2

    tst = pyro.sample(dist.Categorical(pdf_c)) # nx2

TypeError: __init__() missing 1 required positional argument: 'scale'